In [1]:
suppressMessages(require(Rtsne))
suppressMessages(require(cellrangerRkit))
suppressMessages(require(ggplot2))
suppressMessages(require(data.table))
suppressMessages(require(gplots))
suppressMessages(require(matrixStats))
suppressMessages(require(RColorBrewer))
require(RUVnormalize)

Loading required package: RUVnormalize


In [2]:
genome="GRCh38"
#sample_list=c("C12pBMC","C12Rectum","C9pBMC","C9Rectum") #
#cell_assignment_gps=c("failed BCR/TCR","B cell","T cell","dual-label","unknown")
load("testing_RUVscale_mt_unknownRMed.Rdata") #"testing_RUVscale.Rdata"

patient_type=unique(gsub("_.*","",sample_list))
patient_assignment=rep(0,length(sample_gp))
for(i in 1:length(patient_type)){
    patient_assignment[sample_gp %in% grep(patient_type[i],sample_list)]=i
}

tissue_type=c("pBMC","I","R")
tissue_assignment=rep(0,length(sample_gp))
for(i in 1:length(tissue_type)){
    tissue_assignment[sample_gp %in% grep(tissue_type[i],sample_list)]=i
}

In [4]:
save(sample_gp,ExpressionMat,cell_assignment,file="ExpMatRaw_VDJ.Rdata")
ls()

[1] "BCR_annoation"                         
 [2] "BCR_annoation_w"                       
 [3] "cell_assignment"                       
 [4] "cell_assignment_gps"                   
 [5] "center_by_batch"                       
 [6] "cross_combining"                       
 [7] "ercc_matrix"                           
 [8] "ExpressionBinaryMat"                   
 [9] "ExpressionFiltered"                    
[10] "ExpressionMat"                         
[11] "ExpressionNormed"                      
[12] "genome"                                
[13] "housekeeping_gene"                     
[14] "i"                                     
[15] "mart"                                  
[16] "num_of_unknown_cell_at_most_per_lirary"
[17] "patient_assignment"                    
[18] "patient_type"                          
[19] "pca_allcell"                           
[20] "ptm"                                   
[21] "rtsne_allcell"                         
[22] "sample_gp"                             
[23] "sample_list"                           
[24] "sdY"                                   
[25] "second_to_humanReadableTime"           
[26] "ssd"                                   
[27] "t"                                     
[28] "t2g"                                   
[29] "tag_B"                                 
[30] "taggene"                               
[31] "taggene_mt"                            
[32] "taggene_posi"                          
[33] "tag_T"                                 
[34] "TCR_annoation"                         
[35] "TCR_annoation_w"                       
[36] "tissue_assignment"                     
[37] "tissue_type"                           
[38] "valid_BCR_list"                        
[39] "valid_B_list"                          
[40] "valid_cell_barcodes"                   
[41] "valid_sc_barcode"                      
[42] "valid_TCR_list"                        
[43] "valid_T_list"                          
[44] "venn_matrix"

### plot PBMC and Rectum sperately

In [3]:
require(repr)
#options(repr.plot.width=8, repr.plot.height=8)

coltouse=cell_assignment
tagcol=RColorBrewer::brewer.pal(length(cell_assignment_gps),"Set1")
for (i in 1:length(cell_assignment_gps)){
    coltouse[coltouse==cell_assignment_gps[i]]=tagcol[i]
}

pdf("TSNE for PBMC and Rectum seperately.pdf",width=10,height=10)
par(mfrow=c(2,2))
#plot PBMC
tagcell=sample_gp %in%  (grep("pBMC",sample_list))
plot(rtsne_allcell$Y[tagcell,],pch=19,col=coltouse[tagcell],cex=0.5,main="TSNE B/T cell label",xlim=c(min(rtsne_allcell$Y[tagcell,1]),max(rtsne_allcell$Y[tagcell,1])+20),
    xlab="TSNE 1",ylab="TSNE 2")
legend("topright",cell_assignment_gps,bty = "n",lty=0,pch=19,col=tagcol)

tagcol=RColorBrewer::brewer.pal(length(sample_list),"Set1")
plot(rtsne_allcell$Y[tagcell,],pch=19,col=tagcol[sample_gp[tagcell]],cex=0.5,main="TSNE PBMC/Rectum label",xlim=c(min(rtsne_allcell$Y[,1]),max(rtsne_allcell$Y[,1])+10),
    xlab="TSNE 1",ylab="TSNE 2")
legend("topright",sample_list[grep("pBMC",sample_list)],bty = "n",lty=0,pch=19,col=tagcol[grep("pBMC",sample_list)])

#plot Rectumn
tagcell=sample_gp %in%  (grep("_R",sample_list))
plot(rtsne_allcell$Y[tagcell,],pch=19,col=coltouse[tagcell],cex=0.5,main="TSNE B/T cell label",xlim=c(min(rtsne_allcell$Y[tagcell,1]),max(rtsne_allcell$Y[tagcell,1])+20),
    xlab="TSNE 1",ylab="TSNE 2")
legend("topright",cell_assignment_gps,bty = "n",lty=0,pch=19,col=tagcol)

tagcol=RColorBrewer::brewer.pal(length(sample_list),"Set1")
plot(rtsne_allcell$Y[tagcell,],pch=19,col=tagcol[sample_gp[tagcell]],cex=0.5,main="TSNE PBMC/Rectum label",xlim=c(min(rtsne_allcell$Y[,1]),max(rtsne_allcell$Y[,1])+10),
    xlab="TSNE 1",ylab="TSNE 2")
legend("topright",sample_list[grep("_R",sample_list)],bty = "n",lty=0,pch=19,col=tagcol[grep("Rectum",sample_list)])
dev.off()

Loading required package: repr
Warning message in RColorBrewer::brewer.pal(length(sample_list), "Set1"):
“n too large, allowed maximum for palette Set1 is 9
Returning the palette you asked for with that many colors
”Warning message in RColorBrewer::brewer.pal(length(sample_list), "Set1"):
“n too large, allowed maximum for palette Set1 is 9
Returning the palette you asked for with that many colors
”

png 
  2

# clonetype analysis

In [3]:
valid_BCR_list=which(BCR_annoation$is_cell & BCR_annoation$high_confidence & BCR_annoation$productive)
valid_TCR_list=which(TCR_annoation$is_cell & TCR_annoation$high_confidence & TCR_annoation$productive)
recombined_BCR_list=which(is.na(BCR_annoation_w$v_gene)+is.na(BCR_annoation_w$d_gene)+is.na(BCR_annoation_w$j_gene)+is.na(BCR_annoation_w$c_gene)<3)
recombined_TCR_list=which(is.na(TCR_annoation_w$v_gene)+is.na(TCR_annoation_w$d_gene)+is.na(TCR_annoation_w$j_gene)+is.na(TCR_annoation_w$c_gene)<3)
BCR_annoation_w=BCR_annoation_w[recombined_BCR_list,]
TCR_annoation_w=TCR_annoation_w[recombined_TCR_list,]


In [48]:
sample_list

[1] "C9_pBMC"  "C9_R"     "C12_pBMC" "C12_R"    "C17_pBMC" "C17_R"   
 [7] "C17_I"    "C18_pBMC" "C18_R"    "C18_I"

In [4]:
#identify clonetype that's the same chain - v d j gene
T_tag_mat=TCR_annoation[valid_TCR_list,c("barcode","v_gene","d_gene","j_gene","c_gene","cdr3_nt","raw_clonotype_id")]
setDT(T_tag_mat)[, contig_id := paste0("clonetype_",.GRP), by = .(v_gene,d_gene,j_gene,c_gene,cdr3_nt)]
sample_group=as.numeric(vapply(strsplit(T_tag_mat$barcode,"-"), `[`, 2, FUN.VALUE=character(1)))
T_tag_mat$raw_clonotype_id=paste0(T_tag_mat$raw_clonotype_id,"_",sample_group)
expaned_clone=which(table(T_tag_mat$raw_clonotype_id)>1)

plot_table=list()
for(i in 1:max(sample_group)){
    tag=which(sample_group==i)
    tag=tag[!duplicated(T_tag_mat$barcode[tag])]
    tag_id_tbl=table(T_tag_mat$raw_clonotype_id[tag])
    plot_table[[i]]=data.frame(clone_id=names(tag_id_tbl),
                               count=as.vector(tag_id_tbl),
                               freq=as.vector(tag_id_tbl)/sum(tag_id_tbl),
                               groups=sample_list[i]) 
}
T_clonetype_stat=do.call(rbind,plot_table)
T_clonetype_stat=T_clonetype_stat[order(T_clonetype_stat$freq,decreasing = T),]

plot_table=list()
for(i in 1:max(sample_group)){
    tag=T_clonetype_stat$groups==sample_list[i]
    tag_clonetype_stat=T_clonetype_stat[tag,]
    tag_freq_tbl=table(tag_clonetype_stat$freq)
    tag_count_tbl=table(tag_clonetype_stat$count)
    plot_table[[i]]=data.frame(groups=sample_list[i],
                               expansion_level_freq=as.numeric(names(tag_freq_tbl)),
                               expansion_level_count=as.numeric(names(tag_count_tbl)),
                               clone_count=as.vector(tag_count_tbl)) 
}
T_clonetype_expansion_count=do.call(rbind,plot_table)

T_tag_mat=cbind(T_tag_mat,T_clonetype_stat$count[match(T_tag_mat$raw_clonotype_id,T_clonetype_stat$clone_id)]) 
T_tag_mat=cbind(T_tag_mat,T_clonetype_stat$freq[match(T_tag_mat$raw_clonotype_id,T_clonetype_stat$clone_id)])
colnames(T_tag_mat)[9:10]=c("count","freq")

In [5]:
#B cell
B_tag_mat=BCR_annoation[valid_BCR_list,c("barcode","v_gene","d_gene","j_gene","c_gene","cdr3_nt","raw_clonotype_id")]
setDT(B_tag_mat)[, contig_id := paste0("clonetype_",.GRP), by = .(v_gene,d_gene,j_gene,c_gene,cdr3_nt)]
sample_group=as.numeric(vapply(strsplit(B_tag_mat$barcode,"-"), `[`, 2, FUN.VALUE=character(1)))
B_tag_mat$raw_clonotype_id=paste0(B_tag_mat$raw_clonotype_id,"_",sample_group)
expaned_clone=which(table(B_tag_mat$id)>1)

plot_table=list()
for(i in 1:max(sample_group)){
    tag=which(sample_group==i)
    tag=tag[!duplicated(B_tag_mat$barcode[tag])]
    tag_id_tbl=table(B_tag_mat$raw_clonotype_id[tag])
    plot_table[[i]]=data.frame(clone_id=names(tag_id_tbl),
                               count=as.vector(tag_id_tbl),
                               freq=round(as.vector(tag_id_tbl)/sum(tag_id_tbl),4)*100,
                               groups=sample_list[i]) 
}
B_clonetype_stat=do.call(rbind,plot_table)
B_clonetype_stat=B_clonetype_stat[order(B_clonetype_stat$freq,decreasing = T),]

plot_table=list()
for(i in 1:max(sample_group)){
    tag=B_clonetype_stat$groups==sample_list[i]
    tag_clonetype_stat=B_clonetype_stat[tag,]
    tag_freq_tbl=table(tag_clonetype_stat$freq)
    tag_count_tbl=table(tag_clonetype_stat$count)
    plot_table[[i]]=data.frame(groups=sample_list[i],
                               expansion_level_freq=as.numeric(names(tag_freq_tbl)),
                               expansion_level_count=as.numeric(names(tag_count_tbl)),
                               clone_count=as.vector(tag_count_tbl)) 
}
B_clonetype_expansion_count=do.call(rbind,plot_table)

B_tag_mat=cbind(B_tag_mat,B_clonetype_stat$count[match(B_tag_mat$raw_clonotype_id,B_clonetype_stat$clone_id)])
B_tag_mat=cbind(B_tag_mat,B_clonetype_stat$freq[match(B_tag_mat$raw_clonotype_id,B_clonetype_stat$clone_id)])
colnames(B_tag_mat)[9:10]=c("count","freq")

In [7]:
T_percell_expansion_tbl=unique(T_tag_mat[,c("barcode","count","freq")])
#T_percell_expansion_tbl=T_percell_expansion_tbl[,.(count=max(count),freq=max(freq)),by=barcode]
B_percell_expansion_tbl=unique(B_tag_mat[,c("barcode","count","freq")])
#B_percell_expansion_tbl=B_percell_expansion_tbl[,.(count=max(count),freq=max(freq)),by=barcode]

#expanision level per cell
count_BT_tbl=data.frame(T=T_percell_expansion_tbl$count[match(rownames(ExpressionNormed),T_percell_expansion_tbl$barcode)],
                       B=B_percell_expansion_tbl$count[match(rownames(ExpressionNormed),B_percell_expansion_tbl$barcode)])
freq_BT_tbl=data.frame(T=T_percell_expansion_tbl$freq[match(rownames(ExpressionNormed),T_percell_expansion_tbl$barcode)],
                       B=B_percell_expansion_tbl$freq[match(rownames(ExpressionNormed),B_percell_expansion_tbl$barcode)])
count_BT_tbl[is.na(count_BT_tbl)]=0
freq_BT_tbl[is.na(freq_BT_tbl)]=0
expansion_count=rowMaxs(as.matrix(count_BT_tbl))
expansion_freq=rowMaxs(as.matrix(freq_BT_tbl))

In [9]:
table(T_percell_expansion_tbl$count)
table(B_percell_expansion_tbl$count)


   1    2    3    4    5    6    8   11   12   15   20 
3437  476  150   48   30   18   24   11   24   15   20 


   1    2    3    4    5    6    7    8    9   10   11   12   13   17   18   20 
2369  478  195  124   60   48   63   40   36   40   11   12   13   17   18   20 
  22   28   29   30   32   33   44   48   57   70   77   80   99  121  150 
  22   28   29   60   32   33   44   48   57   70   77   80   99  121  150 

In [8]:
table(count_BT_tbl$B)
table(count_BT_tbl$T)


   0    1    2    3    4    5    6    7    8    9   10   11   12   13   17   18 
9280 2145  409  163  102   48   43   51   39   22   28   11    2   13   16   11 
  20   22   28   29   30   32   33   44   48   57   70   77   80   99  121  150 
   9   22    4   29    9    9    7   44    5   56   65    4    9   21   34   14 


   0    1    2    3    4    5    6    8   11   12   15   20 
8471 3437  476  150   48   30   18   24   11   24   15   20 

## plot expansion level by cell

In [10]:
#require(repr)
#options(repr.plot.width=12, repr.plot.height=4)
pdf("expansion level vs. RNA TSNE.pdf",width=15,height=5)
par(mfrow=c(1,3))

coltouse=cell_assignment
tagcol=RColorBrewer::brewer.pal(length(cell_assignment_gps),"Set1")
for (i in 1:length(cell_assignment_gps)){
    coltouse[coltouse==cell_assignment_gps[i]]=tagcol[i]
}
plot(rtsne_allcell$Y,pch=19,col=coltouse,cex=0.5,main="all cells by celltype",xlim=c(min(rtsne_allcell$Y),max(rtsne_allcell$Y)+20),
    xlab="TSNE 1",ylab="TSNE 2")
legend("topright",cell_assignment_gps,bty = "n",lty=0,pch=19,col=tagcol)

rbPal <- colorRampPalette(RColorBrewer::brewer.pal(9,"Reds"))
expansion_count[expansion_count>50]=50
breaks=seq(min(expansion_count),max(expansion_count),length.out=25)
cols=rbPal(25)[as.numeric(cut(expansion_count,breaks = breaks, include.lowest=TRUE))]
#plot(rtsne_allcell$Y[expansion_count<=1,],pch=19,col="gray",cex=0.5,main="all cells by expansion count",xlim=c(min(rtsne_allcell$Y),max(rtsne_allcell$Y)+20),
#    xlab="TSNE 1",ylab="TSNE 2")
plot(rtsne_allcell$Y[expansion_count>0,],pch=19,col=cols[expansion_count>0],cex=0.5,main="count")

expansion_freq[expansion_freq>5]=5
breaks=seq(min(expansion_freq)-1,max(expansion_freq),length.out=25)
cols=rbPal(25)[as.numeric(cut(expansion_freq,breaks = breaks, include.lowest=TRUE))]
#plot(rtsne_allcell$Y[expansion_count<=1,],pch=19,col="gray",cex=0.5,main="all cells by expansion freq",xlim=c(min(rtsne_allcell$Y),max(rtsne_allcell$Y)+20),
#    xlab="TSNE 1",ylab="TSNE 2")
plot(rtsne_allcell$Y[expansion_count>0,],pch=19,col=cols[expansion_count>0],cex=0.5,main="freq")
dev.off()

png 
  2

## plot the most expaned several clone

In [32]:

tagcell=(sample_gp %in% grep("C12Rectum",sample_list)) #& (cell_assignment == "T cell")
tagcoord=rtsne_allcell$Y[tagcell,]

require(repr)
#options(repr.plot.width=12, repr.plot.height=4)
pdf("expansion in C12Rectum.pdf",width=15,height=5)
par(mfrow=c(1,3))
plot(tagcoord,pch=19,col=coltouse[tagcell],cex=0.5,main="all cells in C12Rectum",xlim=c(min(rtsne_allcell$Y[tagcell,1]),max(rtsne_allcell$Y[tagcell,1])+20),
    xlab="TSNE 1",ylab="TSNE 2")
legend("topright",cell_assignment_gps,bty = "n",lty=0,pch=19,col=tagcol)

#T cell
tagcell=(sample_gp %in% grep("C12Rectum",sample_list)) & (cell_assignment == "T cell")
tagcoord=rtsne_allcell$Y[tagcell,]
colvector=rep(1,length(tagcell))
labeltouse=c("others")
for(i in 1:3){
    tag_clonetype=T_clonetype_stat$clone_id[T_clonetype_stat$groups=="C12Rectum"][i]
    tag_bc=T_tag_mat$barcode[T_tag_mat$id %in% tag_clonetype]
    tag_cell=which(rownames(ExpressionNormed)[tagcell] %in% tag_bc)
    colvector[tag_cell]=i+1
    labeltouse=c(labeltouse,as.character(tag_clonetype))
}
colvector=c("grey",RColorBrewer::brewer.pal(3,"Set1"))[colvector]

plot(tagcoord,pch=19,col=colvector,cex=0.5,main="top 3 expanded clonetype in T cell",
     xlim=c(min(rtsne_allcell$Y[,1]),max(rtsne_allcell$Y[,1])+20),ylim=c(min(rtsne_allcell$Y[,2]),max(rtsne_allcell$Y[,2])),
    xlab="TSNE 1",ylab="TSNE 2")
legend("topright",labeltouse,bty = "n",lty=0,pch=19,col=c("grey",RColorBrewer::brewer.pal(3,"Set1")))

#B cell
tagcell=(sample_gp %in% grep("C12Rectum",sample_list)) & (cell_assignment == "B cell")
tagcoord=rtsne_allcell$Y[tagcell,]
colvector=rep(1,length(tagcell))
labeltouse=c("others")
for(i in 1:3){
    tag_clonetype=B_clonetype_stat$clone_id[B_clonetype_stat$groups=="C12Rectum"][i]
    tag_bc=B_tag_mat$barcode[B_tag_mat$id %in% tag_clonetype]
    tag_cell=which(rownames(ExpressionMat)[tagcell] %in% tag_bc)
    colvector[tag_cell]=i+1
    labeltouse=c(labeltouse,as.character(tag_clonetype))
}
colvector=c("grey",RColorBrewer::brewer.pal(3,"Set1"))[colvector]

plot(tagcoord,pch=19,col=colvector,cex=0.5,main="top 3 expanded clonetype in B cell",
     xlim=c(min(rtsne_allcell$Y[,1]),max(rtsne_allcell$Y[,1])+20),ylim=c(min(rtsne_allcell$Y[,2]),max(rtsne_allcell$Y[,2])),
    xlab="TSNE 1",ylab="TSNE 2")
legend("topright",labeltouse,bty = "n",lty=0,pch=19,col=c("grey",RColorBrewer::brewer.pal(3,"Set1")))
dev.off()

png 
  2

In [12]:
c(labeltouse,tag_clonetype)

[1] "not expanded" "1287"         "1285"         "1286"         "1286"

In [13]:
tag_clonetype

[1] clonetype_609
2094 Levels: clonetype_1 clonetype_10 clonetype_100 ... clonetype_2094

In [9]:
clonetype_stat[1:5,]

,clone_id,count,freq,groups
1310,clonetype_610,51,0.03233989,C12Rectum
1308,clonetype_608,39,0.02473050,C12Rectum
1718,clonetype_1693,3,0.02362205,C9pBMC
1309,clonetype_609,35,0.02219404,C12Rectum
1704,clonetype_1679,2,0.01574803,C9pBMC


In [38]:
counttbl[counttbl$countTRA>=2,][1:5,]

barcode,countTRA,countTRB,countother
AGTGTCACAGGATTGG-1,2,1,0
CCGTACTGTAGCAAAT-1,2,1,0
CTGCGGAAGGAGTCTG-1,2,1,0
GTACTTTAGCTAGGCA-1,2,1,0
GTCGTAAGTTCTGAAC-1,2,1,0


In [40]:
tagTCR[tagTCR$barcode=="AGTGTCACAGGATTGG-1",]

,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,full_length,productive,cdr3,cdr3_nt,reads,umis,raw_clonotype_id,raw_consensus_id
118,AGTGTCACAGGATTGG-1,TRUE,AGTGTCACAGGATTGG-1_contig_1,TRUE,657,TRB,TRBV6-1,TRBD2,TRBJ2-3,TRBC2,TRUE,TRUE,CASSGLAGYTDTQYF,TGTGCCAGCAGTGGACTAGCGGGGTACACAGATACGCAGTATTTT,37389,13,clonotype14,clonotype14_consensus_2
119,AGTGTCACAGGATTGG-1,TRUE,AGTGTCACAGGATTGG-1_contig_2,TRUE,741,TRA,TRAV12-1,NA,TRAJ31,TRAC,TRUE,TRUE,CVVNLKEARLMF,TGTGTGGTGAACCTGAAGGAGGCCAGACTCATGTTT,32733,8,clonotype14,clonotype14_consensus_1
120,AGTGTCACAGGATTGG-1,TRUE,AGTGTCACAGGATTGG-1_contig_3,TRUE,744,TRA,TRAV12-3,NA,TRAJ13,TRAC,TRUE,TRUE,CAMSAGYQKVTF,TGTGCAATGAGCGCTGGTTACCAGAAAGTTACCTTT,17387,6,clonotype14,clonotype14_consensus_3


In [5]:
tagTCR=TCR_annoation[TCR_annoation$productive & TCR_annoation$high_confidence,]
tagTCR=tagTCR[!is.na(tagTCR$productive),]
dtTCR=data.table(tagTCR)
counttbl=dtTCR[,.(countTRA=sum(chain=="TRA"),countTRB=sum(chain=="TRB"),
                  countTRG=sum(chain=="TRG"),countTRD=sum(chain=="TRD"),
                  countother=sum(chain!="TRB" &chain!="TRA" &chain!="TRD" &chain!="TRG")),by=barcode]

In [9]:
unique(dtTCR$chain)
length(unique(dtTCR$barcode))

[1] "TRB"   "TRA"   "Multi" "TRD"

[1] 4253

In [6]:
table(counttbl$countTRG,counttbl$countTRD)

   
       0    1
  0 4252    1

In [24]:
table(counttbl$countTRA,counttbl$countTRB)

   
       0    1    2    3
  0    1  365  186    8
  1   29 2123  766   34
  2   13  412  241   14
  3    0   10   18    0

In [32]:
tagBCR=BCR_annoation[BCR_annoation$productive & BCR_annoation$high_confidence,]
tagBCR=tagBCR[!is.na(tagBCR$productive),]
dtBCR=data.table(tagBCR)
counttbl=dtBCR[,.(countIGL=sum(chain=="IGL"),countIGH=sum(chain=="IGH"),countIGK=sum(chain=="IGK"),countMulti=sum(chain=="Multi")),by=barcode]

In [25]:
BCR_annoation[1:5,]

,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,full_length,productive,cdr3,cdr3_nt,reads,umis,raw_clonotype_id,raw_consensus_id
1,AAGGTTCTCACATGCA-1,TRUE,AAGGTTCTCACATGCA-1_contig_1,TRUE,682,IGL,IGLV3-10,NA,IGLJ3,IGLC2,TRUE,TRUE,CYSTDSSGNHWVF,TGTTACTCAACAGACAGCAGTGGTAATCATTGGGTGTTC,4444,22,clonotype6,clonotype6_consensus_2
2,AAGGTTCTCACATGCA-1,TRUE,AAGGTTCTCACATGCA-1_contig_2,FALSE,788,IGL,IGLV3-10,NA,IGLJ3,IGLC2,FALSE,NA,CYSTDSSGNHWVF,TGTTACTCAACAGACAGCAGTGGTAATCATTGGGTGTTC,362,2,clonotype6,NA
3,AAGGTTCTCACATGCA-1,TRUE,AAGGTTCTCACATGCA-1_contig_3,TRUE,535,IGH,IGHV3-33,IGHD4-17,IGHJ1,IGHM,TRUE,TRUE,CALGDSRDQNW,TGTGCGCTCGGTGACTCCCGAGACCAGAACTGG,177,4,clonotype6,clonotype6_consensus_1
5,ACACCGGAGATGTAAC-1,TRUE,ACACCGGAGATGTAAC-1_contig_2,TRUE,577,IGK,IGKV1D-39,NA,IGKJ3,IGKC,TRUE,TRUE,CQQSYSTLFTF,TGTCAACAGAGTTACAGTACCTTATTCACTTTC,411,2,clonotype7,clonotype7_consensus_1
6,ACACCGGAGATGTAAC-1,TRUE,ACACCGGAGATGTAAC-1_contig_3,TRUE,440,IGH,IGHV3-7,NA,NA,NA,FALSE,NA,NA,NA,126,1,clonotype7,NA


In [33]:
table(counttbl$countIGH,counttbl$countIGL)
table(counttbl$countIGH,counttbl$countIGK)
table(counttbl$countIGK,counttbl$countIGL)

   
       0    1    2
  0  624  552    0
  1 1841 1259   34
  2   27   98   56
  3    0    3    0

   
       0    1    2    3
  0  459  709    8    0
  1 1370 1746   16    2
  2   87   82   12    0
  3    0    0    3    0

   
       0    1    2
  0  318 1524   74
  1 2140  381   16
  2   33    6    0
  3    1    1    0

In [31]:
dtBCR[1:5,]

barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,full_length,productive,cdr3,cdr3_nt,reads,umis,raw_clonotype_id,raw_consensus_id
AACACGTCAAGTCATC-1,TRUE,AACACGTCAAGTCATC-1_contig_1,TRUE,699,TRB,TRBV7-9,TRBD1,TRBJ2-7,TRBC2,TRUE,TRUE,CASSSTPGQGNYEQYF,TGTGCCAGCAGCTCTACGCCGGGACAGGGTAACTACGAGCAGTACTTC,80699,28,clonotype6,clonotype6_consensus_1
AACACGTCAAGTCATC-1,TRUE,AACACGTCAAGTCATC-1_contig_3,TRUE,617,TRA,TRAV8-6,NA,TRAJ37,TRAC,TRUE,TRUE,CAADSGNTGKLIF,TGTGCTGCCGACTCTGGCAACACAGGCAAACTAATCTTT,6705,3,clonotype6,clonotype6_consensus_2
AACTCAGTCTGATACG-1,TRUE,AACTCAGTCTGATACG-1_contig_1,TRUE,760,TRB,TRBV19,TRBD2,TRBJ1-1,TRBC1,TRUE,TRUE,CASSSGRQAEAFF,TGTGCCAGTAGCTCGGGACGTCAAGCTGAAGCTTTCTTT,12429,4,clonotype1,clonotype1_consensus_1
AACTCAGTCTGATACG-1,TRUE,AACTCAGTCTGATACG-1_contig_14,TRUE,332,TRG,TRGV10,NA,NA,NA,FALSE,FALSE,CPHLCTWLSRPILLEL*W,TGCCCTCATCTTTGCACTTGGCTTTCTAGGCCCATCCTCTTGGAGCTGTAGTGG,35,1,clonotype1,NA
AACTCAGTCTGATACG-1,TRUE,AACTCAGTCTGATACG-1_contig_31,TRUE,351,Multi,TRBV20-1,TRBD1,TRBJ2-1,IGHG1,FALSE,NA,NA,NA,55,1,clonotype1,NA
